In [21]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../../../data_analysis/')
from simulate_door_slam import *
from trajectory import *
from utils import *
from local_plot import *

def read_frame_timestamps(g2o_path, nodes):
    frame_id_to_stamp = {}
    for i in nodes:
        with open(f"{g2o_path}/frame_timestamp_{i}.txt", "r") as f:
            for line in f:
                frame_id, stamp = line.split()
                frame_id_to_stamp[int(frame_id)] = float(stamp)
    return frame_id_to_stamp

def process_output(g2o_path, nodes):
    poses_pgo, t0 = read_paths(g2o_path, nodes, prefix="pgo_")
    paths_gt, _ = read_paths(g2o_path + "/../", nodes, prefix="groundtruth_", t0=t0)
    align_paths(poses_pgo, paths_gt, align_by_first=True, align_with_minize=True)
    display(plot_fused_err(nodes, poses_pgo, paths_gt, show=False))
    # plot_fused(nodes, poses_pgo, paths_gt)
    dte = 550
    display(relative_pose_err(nodes, poses_pgo, paths_gt, dte=dte))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Note
We need to start ROS Core Manually for this file

In [22]:
def evaluate_pgo(g2o_path, g2o_input, max_steps=100, eta_k=1.45, rho_frame_T=0.39, 
            rho_frame_theta=1.556, simulate_delay_ms=0.0, verify_accuaracy=False, agent_num=5, dgs_max_steps=50, 
            max_solving_time=10.0, rho_rot_mat=0.09):
    pg = PoseGraph(g2o_input, prt=False)
    ignore_infor = False
    print(f"Initial cost: {pg.evaluate_cost():.1f} edges {len(pg.edges)}")
    if verify_accuaracy:
        frame_id_to_stamp = read_frame_timestamps(g2o_path, range(agent_num))
    output_path=g2o_path + "/d2pgo-rot-inited/"
    pgo_optimized, max_solve_time, mean_iters = call_d2pgo_opti(g2o_folder=g2o_input, output_folder=output_path, enable_rot_init=True, 
        max_steps=max_steps, agent_num=agent_num, ignore_infor=ignore_infor, eta_k=eta_k, rho_frame_theta=rho_frame_theta, 
        rho_frame_T=rho_frame_T, rho_rot_mat=rho_rot_mat, simulate_delay_ms=simulate_delay_ms, max_solving_time=max_solving_time)
    if verify_accuaracy:
        pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
        process_output(output_path, [0, 1, 2, 3, 4])
    print("Avg Traj len: ", pgo_optimized.evaluate_trajectory_length())
    print(f"D2PGO RotInit cost: {pgo_optimized.evaluate_cost():.1f} solve_time {max_solve_time:.1f} ms iters {mean_iters}")

    output_path=g2o_path + "/d2pgo-no-rot-init/"
    pgo_optimized, max_solve_time, mean_iters  = call_d2pgo_opti(g2o_folder=g2o_input, output_folder=output_path, enable_rot_init=False, 
        max_steps=max_steps, agent_num=agent_num, ignore_infor=ignore_infor, eta_k=eta_k, rho_frame_theta=rho_frame_theta, 
        rho_frame_T=rho_frame_T, rho_rot_mat=rho_rot_mat, simulate_delay_ms=simulate_delay_ms, max_solving_time=max_solving_time)
    print(f"D2PGO without RotInit cost: {pgo_optimized.evaluate_cost():.1f} solve_time {max_solve_time:.1f} ms iters {mean_iters}")
    if verify_accuaracy:
        pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
        process_output(output_path, [0, 1, 2, 3, 4])
    output_path = g2o_path + "/DGS/"
    iterations, min_time, max_time, initial, final, util_rate, total_optim = call_DGS_solver(g2o_input, agent_num=agent_num, 
        rthresh=1e-4, pthresh=1e-4, maxIter=dgs_max_steps, between_noise="true")
    pgo_optimized = copy.copy(pg)
    pgo_optimized.read_g2o_single(g2o_input+"/fullGraph_optimized.g2o", update_only=True)
    total_time = max_time + iterations*simulate_delay_ms #Assume communication time is 100ms
    print(f"DGS cost: {pgo_optimized.evaluate_cost():.1f} max_time: {max_time} iter {iterations} total_time {total_time} DGS cost initial {initial} final: {final} ")
    if verify_accuaracy:
        pgo_optimized.write_to_csv(output_path, frame_id_to_stamp)
        process_output(output_path, [0, 1, 2, 3, 4])


In [28]:
g2o_path="/home/xuhao/data/d2slam/pgo/tum_corr_5/"
g2o_input = g2o_path + "/input/"
max_steps = 1000
eta_k=1.5101010101010102
rho_frame_T=0.4526572657265727
rho_frame_theta=2.868058805880588
rho_rot_mat =0.0918787878787879
simulate_delay_ms=0.0
max_solving_time =20.0

evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat, dgs_max_steps=40)

Total agents 5 keyframes 12065 edges 16814 inter edge 3165 comm_vol 4754 keyframes 759<->3030
Initial cost: 350408.6 edges 16814
Avg Traj len:  255.83023351591382
D2PGO RotInit cost: 61.6 solve_time 20044.6 ms iters 227.0
D2PGO without RotInit cost: 66339.5 solve_time 20030.2 ms iters 387.0
DGS cost: 59.0 max_time: 1566.2 iter 48 total_time 1566.2 DGS cost initial 236205.1 final: 72.7988 


In [11]:
g2o_path="/home/xuhao/data/d2slam/pgo/parking-garage/"
g2o_input = g2o_path + "/input/"
max_steps = 500
evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat)

Total agents 5 keyframes 1661 edges 6275 inter edge 3246 comm_vol 1673 keyframes 167<->498
Initial cost: 1239528.0 edges 6275
Avg Traj len:  1404.7468635104265
D2PGO RotInit cost: 18.3 solve_time 22397.4 ms iters 500.0
D2PGO without RotInit cost: 155583.7 solve_time 12452.9 ms iters 500.0
DGS cost: 0.7 max_time: 1111.2 iter 100 total_time 6111.2 DGS cost initial 1227785.1 final: 0.69 


In [30]:
g2o_path="/home/xuhao/data/d2slam/pgo/cubicle/"
g2o_input = g2o_path + "/input/"
evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat)

Total agents 5 keyframes 5750 edges 12486 inter edge 1532 comm_vol 2427 keyframes 576<->1725
Initial cost: 32260989610.7 edges 12486
Avg Traj len:  289.83567125921047
D2PGO RotInit cost: 194050320.6 solve_time 10463.8 ms
D2PGO without RotInit cost: 371158668.4 solve_time 10533.0 ms
DGS cost: 5514448.0 max_time: 2636.4 iter 100 total_time 7636.4 DGS cost initial 18010192.4 final: 1365.8682 


In [17]:
g2o_path="/home/xuhao/data/d2slam/pgo/OmniLong5Yaw/"
g2o_input = g2o_path + "/input/"
max_solving_time = 20
evaluate_pgo(g2o_path, g2o_input, max_steps, eta_k, rho_frame_T, rho_frame_theta, 
                simulate_delay_ms, max_solving_time=max_solving_time, rho_rot_mat=rho_rot_mat)

Total agents 5 keyframes 16430 edges 51358 inter edge 19738 comm_vol 27094 keyframes 3010<->3375
Initial cost: 520424.1 edges 51358
Avg Traj len:  242.8251669963974
D2PGO RotInit cost: 1365.7 solve_time 20382.5 ms iters 37.0
D2PGO without RotInit cost: 40521.1 solve_time 20480.3 ms iters 32.0
DGS cost: 959.0 max_time: 4262.8 iter 52 total_time 6862.8 DGS cost initial 589298.9 final: 1815.0651 


In [58]:
pg = PoseGraph(g2o_input)
pg.write_to_g2o(g2o_input+"/../fullGraph.g2o")
output_path = g2o_path + "/d2pgo-single/"
pgo_optimized, max_solve_time, mean_iters = call_d2pgo_opti(g2o_folder=g2o_input+"/../fullGraph.g2o", output_folder=output_path, enable_rot_init=True, 
        max_steps=max_steps, max_solving_time=max_solving_time, is_single_mode=True)
print(f"D2PGO RotInit cost: {pgo_optimized.evaluate_cost():.1f} solve_time {max_solve_time:.1f} ms iters {mean_iters}")
# pg.show("raw", show_edges=False)
# pgo_optimized.show("opti")

roslaunch d2pgo d2pgo_test_single.launch g2o_path:=/home/xuhao/data/d2slam/pgo/tum_corr_5//input//../fullGraph.g2o             output_path:=/home/xuhao/data/d2slam/pgo/tum_corr_5//d2pgo-single/ enable_rot_init:=True ignore_infor:=False             enable_linear_pose6d_solver:=False solver_type:='ceres'
D2PGO RotInit cost: 8539.5 solve_time 13206.2 ms iters 1.0
